In [1]:
import os
import json 
import pandas as pd
import traceback

In [3]:
from langchain.chat_models import ChatOpenAI

In [5]:
from dotenv import load_dotenv

load_dotenv()


True

In [6]:
KEY=os.getenv("OPENAI_API_KEY")

In [8]:
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo", temperature=0.5)

c:\Users\user\mcqgen\env\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [9]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000186C1777E50>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000186C1785CD0>, temperature=0.5, openai_api_key='sk-proj-jRl97zFvs9fLztU1Dvl9T3BlbkFJdRuyxpKZgUM5w0tgS3Et', openai_proxy='')

In [4]:
# Let's test the model with a question-answering prompt.
test_prompt = "What is the capital of France?"

# Generate a response using the ChatLlama instance.
response = llm.generate_response(test_prompt)
print(response)


london


In [5]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [6]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [7]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [8]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [9]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

c:\Users\user\mcqgen\env\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


ValidationError: 2 validation errors for LLMChain
llm
  instance of Runnable expected (type=type_error.arbitrary_type; expected_arbitrary_type=Runnable)
llm
  instance of Runnable expected (type=type_error.arbitrary_type; expected_arbitrary_type=Runnable)

In [15]:
class ChatLlama:
    def __init__(self, model_name, temperature=0.7):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
        self.temperature = temperature

    def generate_response(self, prompt, max_length=50):
        input_ids = self.tokenizer(prompt, return_tensors='pt').input_ids
        outputs = self.model.generate(
            input_ids,
            max_length=max_length,
            do_sample=True,
            temperature=self.temperature
        )
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

# Define the prompt templates for quiz generation and evaluation
quiz_generation_prompt_template = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""

quiz_evaluation_prompt_template = """
You are an expert English grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. 
Only use at max 50 words for complexity analysis. 
If the quiz is not at par with the cognitive and analytical abilities of the students,\
update the quiz questions which need to be changed and change the tone such that it perfectly fits the student abilities.
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

# Define the prompt templates
quiz_generation_prompt = PromptTemplate(input_variables=["text", "number", "subject", "tone", "response_json"],
                                        template=quiz_generation_prompt_template)

quiz_evaluation_prompt = PromptTemplate(input_variables=["subject", "quiz"],
                                        template=quiz_evaluation_prompt_template)

# Create the LLMChain instances
quiz_chain = LLMChain(llm=ChatLlama(model_name="google/flan-t5-base", temperature=0.5), 
                      prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

review_chain = LLMChain(llm=ChatLlama(model_name="google/flan-t5-base", temperature=0.5), 
                        prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

generate_evaluate_chain = SequentialChain(chains=[quiz_chain, review_chain], 
                                          input_variables=["text", "number", "subject", "tone", "response_json"],
                                          output_variables=["quiz", "review"], verbose=True)


ValidationError: 2 validation errors for LLMChain
llm
  instance of Runnable expected (type=type_error.arbitrary_type; expected_arbitrary_type=Runnable)
llm
  instance of Runnable expected (type=type_error.arbitrary_type; expected_arbitrary_type=Runnable)

In [19]:
quiz_generation_template = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""

quiz_evaluation_template = """
You are an expert English grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

# Define the function to generate and evaluate the quiz
def generate_and_evaluate_quiz(text, number, subject, tone, response_json):
    # Generate quiz
    quiz_generation_prompt = quiz_generation_template.format(
        text=text,
        number=number,
        subject=subject,
        tone=tone,
        response_json=response_json
    )
    
    quiz = pipeline("text2text-generation", model="google/flan-t5-base", task="text-generation")(quiz_generation_prompt)[0]['generated_text']
    
    # Evaluate quiz
    quiz_evaluation_prompt = quiz_evaluation_template.format(
        subject=subject,
        quiz=quiz
    )
    
    review = pipeline("text2text-generation", model="google/flan-t5-base", task="text-generation")(quiz_evaluation_prompt)[0]['generated_text']
    
    return quiz, review

# Example usage
text = "Lorem ipsum dolor sit amet, consectetur adipiscing elit."
number = 3
subject = "English"
tone = "neutral"
response_json = RESPONSE_JSON  # Assuming RESPONSE_JSON is defined

quiz, review = generate_and_evaluate_quiz(text, number, subject, tone, response_json)
print("Generated Quiz:")
print(quiz)
print("\nReview:")
print(review)

TypeError: pipeline() got multiple values for argument 'task'

In [17]:
from transformers import pipeline

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from langchain.llms import T5LLM
from langchain.chains import LLMChain, SequentialChain

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

# Define the PromptTemplate for quiz generation
TEMPLATE = """
Text: {text}
You are an expert MCQ maker. Given the above text, it is your job to create a quiz of {number} multiple choice questions for {subject} students in {tone} tone. Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""

quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

# Define the PromptTemplate for quiz evaluation
TEMPLATE2 = """
You are an expert English grammarian and writer. Given a Multiple Choice Quiz for {subject} students. You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. if the quiz is not at per with the cognitive and analytical abilities of the students, update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject", "quiz"],
    template=TEMPLATE2
)

# Initialize T5LLM
llm = T5LLM(tokenizer=tokenizer, model=model)

# Define LLMChain for quiz generation and evaluation
quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

# Define SequentialChain for combined generation and evaluation
generate_evaluate_chain = SequentialChain(
    chains=[quiz_chain, review_chain],
    input_variables=["text", "number", "subject", "tone", "response_json"],
    output_variables=["quiz", "review"],
    verbose=True,
)


ImportError: cannot import name 'T5LLM' from 'langchain.llms' (c:\Users\user\mcqgen\env\lib\site-packages\langchain\llms\__init__.py)

In [2]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [24]:
pip install sentencepiece

Note: you may need to restart the kernel to use updated packages.
